In [1]:
import pandas as pd
import pickle
import numpy as np
from typing import Tuple
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
# baseline model
def build_model():
	# create model
	model = Sequential()
	model.add(Dense(24, input_dim=24, activation='relu'))
	model.add(Dense(48, activation='relu'))
	model.add(Dense(24, activation='relu'))
	model.add(Dense(12, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
		
	return model


In [3]:
accuracyDL= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
metricsDL_buecher0= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_buecher1= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_buecher2= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_buecher3= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_bedeutung1= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_bedeutung2= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_bedeutung3= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])
metricsDL_bedeutung4= pd.DataFrame(columns=['Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC'])

n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']
#
for i in n:
    path='matrices_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()

    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    model = build_model()

    model.compile(
        loss="binary_crossentropy",
        optimizer='Adam',
        metrics=["accuracy"]
    )

    model.fit(
        x=X_train,
        y=y_train,
        epochs=10,
        batch_size=128,
        verbose=0,
        validation_data=(X_test, y_test)
    )

    scores = model.evaluate(
        x=X_test,
        y=y_test,
        verbose=0
    )

    yhat_probs = model.predict(X_test, verbose=0)
    yhat_classes =  (model.predict(X_test) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y_test, yhat_classes)
    precision = precision_score(y_test, yhat_classes)
    recall = recall_score(y_test, yhat_classes)
    f1 = f1_score(y_test, yhat_classes)
    auc = roc_auc_score(y_test, yhat_probs)
    
    accuracyDL = accuracyDL.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)
    accuracyDL.to_pickle('accuracyDL.pkl')

    ## Bedeutung Noten
    path='matrices_bedeutung1/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_bedeutung1 = metricsDL_bedeutung1.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)

    path='matrices_bedeutung2/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_bedeutung2 = metricsDL_bedeutung2.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)


    path='matrices_bedeutung3/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_bedeutung3 = metricsDL_bedeutung3.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)


    path='matrices_bedeutung4/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_bedeutung4 = metricsDL_bedeutung4.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)


    #Buecher
    path='matrices_buecher_0/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_buecher0 = metricsDL_buecher0.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)

    path='matrices_buecher_1/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_buecher1 = metricsDL_buecher1.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)


    path='matrices_buecher_2/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_buecher2 = metricsDL_buecher2.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)


    path='matrices_buecher_3/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')

    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y, yhat_classes)
    precision = precision_score(y, yhat_classes)
    recall = recall_score(y, yhat_classes)
    f1 = f1_score(y, yhat_classes)
    auc = roc_auc_score(y, yhat_probs)
    metricsDL_buecher3 = metricsDL_buecher3.append({'Sentence': i, 'Accuracy':accuracy,'Precision': precision, 'Recall':recall, 'F1':f1, 'AUC':auc}, ignore_index=True)

   

In [4]:
metricsDL_buecher0 = metricsDL_buecher0.add_prefix('buecher0_')
metricsDL_buecher0 = metricsDL_buecher0.rename(columns = {'buecher0_Sentence':'Sentence'}, inplace = False)
metricsDL_buecher1 = metricsDL_buecher1.add_prefix('buecher1_')
metricsDL_buecher1 = metricsDL_buecher1.rename(columns = {'buecher1_Sentence':'Sentence'}, inplace = False)
metricsDL_buecher2 = metricsDL_buecher2.add_prefix('buecher2_')
metricsDL_buecher2 = metricsDL_buecher2.rename(columns = {'buecher2_Sentence':'Sentence'}, inplace = False)
metricsDL_buecher3 = metricsDL_buecher3.add_prefix('buecher3_')
metricsDL_buecher3 = metricsDL_buecher3.rename(columns = {'buecher3_Sentence':'Sentence'}, inplace = False)


metrics_buecher = pd.merge(metricsDL_buecher1, metricsDL_buecher2, on='Sentence')
metrics_buecher = pd.merge(metrics_buecher, metricsDL_buecher3, on='Sentence')
metrics_buecher = pd.merge(metrics_buecher, metricsDL_buecher0, on='Sentence')

metrics_buecher.to_excel('metrics_buecherDL.xlsx')


In [5]:
metricsDL_bedeutung1 = metricsDL_bedeutung1.add_prefix('bedeutung1_')
metricsDL_bedeutung1 = metricsDL_bedeutung1.rename(columns = {'bedeutung1_Sentence':'Sentence'}, inplace = False)
metricsDL_bedeutung2 = metricsDL_bedeutung2.add_prefix('bedeutung2_')
metricsDL_bedeutung2 = metricsDL_bedeutung2.rename(columns = {'bedeutung2_Sentence':'Sentence'}, inplace = False)
metricsDL_bedeutung3 = metricsDL_bedeutung3.add_prefix('bedeutung3_')
metricsDL_bedeutung3 = metricsDL_bedeutung3.rename(columns = {'bedeutung3_Sentence':'Sentence'}, inplace = False)
metricsDL_bedeutung4 = metricsDL_bedeutung4.add_prefix('bedeutung4_')
metricsDL_bedeutung4 = metricsDL_bedeutung4.rename(columns = {'bedeutung4_Sentence':'Sentence'}, inplace = False)


metrics_bedeutung = pd.merge(metricsDL_bedeutung1, metricsDL_bedeutung2, on='Sentence')
metrics_bedeutung = pd.merge(metrics_bedeutung, metricsDL_bedeutung3, on='Sentence')
metrics_bedeutung = pd.merge(metrics_bedeutung, metricsDL_bedeutung4, on='Sentence')

metrics_bedeutung.to_excel('metrics_bedeutungDL.xlsx')